In [1]:
import pandas as pd
from dataclasses import dataclass
from typing import List, Set, Optional, Any
from scipy.sparse import random
from scipy import stats
from numpy.random import default_rng


import numpy as np

In [2]:
@dataclass
class Guest:
    name: str
    
@dataclass
class Seat:
    name: str
    
@dataclass
class Chromosome:
    genes: np.array

    @staticmethod
    def get_next_cycle(p1, p2, covered) -> Optional[Set[int]]:
        idx = None
        cycle = set()
        for i in range(len(p1)):
            if p1[i] not in covered:
                idx = i
                break
        if idx is None:
            return None
        cycle.add(p1[idx])
        while True:
            val = p2[idx]
            if val in cycle:
                break
            cycle.add(val)
            idx = np.where(p1 == val)[0][0]
        return cycle
    
    def random(n: int) -> "Chromosome":
        genes = np.array(range(1, n+1)).astype(np.uint8)
        np.random.shuffle(genes)
        return Chromosome(genes)
    
    def mate(self, other: "Chromosome", mutate=False):
        assert len(self.genes) == len(other.genes)
        covered = set()
        offspring_1, offspring_2 = np.zeros(len(self.genes)).astype(np.uint8), np.zeros(len(self.genes)).astype(np.uint8)
        cycle_idx = 0
        while True:
            cycle = self.get_next_cycle(self.genes, other.genes, covered)
            if cycle is None:
                break
            covered |= cycle
            for i in range(len(self.genes)):
                copy_from_1 = self.genes if cycle_idx % 2 == 0 else other.genes
                copy_from_2 = other.genes if cycle_idx % 2 == 0 else self.genes
                if copy_from_1[i] in cycle:
                    offspring_1[i] = copy_from_1[i]
                if copy_from_2[i] in cycle:
                    offspring_2[i] = copy_from_2[i]
            cycle_idx += 1
        if mutate:
            ix1, ix2 = np.random.randint(0, len(offspring_1)), np.random.randint(0, len(offspring_1))
            offspring_1[ix1], offspring_1[ix2] = offspring_1[ix2], offspring_1[ix1]
            offspring_2[ix1], offspring_2[ix2] = offspring_2[ix2], offspring_2[ix1]
        return Chromosome(offspring_1), Chromosome(offspring_2)
    
    def __eq__(self, other: "Chromosome"):
        return np.all(self.genes == other.genes)
    
    def __hash__(self):
        return hash(str(self.genes.data))
    
@dataclass
class Affinity:
    objects: List[Any]
    matrix: np.array
    
    def random(objects: List[Any]) -> "Affinity":
        rvs = stats.poisson(25, loc=10).rvs
        S = random(len(objects), len(objects), density=0.06, random_state=default_rng(), data_rvs=stats.poisson(25, loc=10).rvs)
        return Affinity(objects, S.A)

    def zeros(objects: List[Any]) -> "Affinity":
        return Affinity(objects, np.zeros((len(objects), len(objects))))

    def set_affinity(self, o1, o2, val, bi_directional=False):
        i = self.objects.index(o1)
        j = self.objects.index(o2)
        self.matrix[i][j] = val
        if bi_directional:
            self.matrix[j][i] = val
    
def fitness(solution: Chromosome, guest_affinity: Affinity, seat_affinity: Affinity):
    score = 0
    assignment = np.zeros((len(seat_affinity.objects), len(guest_affinity.objects)))
    for i, j in enumerate(solution.genes):
        assignment[i][j-1] = 1
    return np.sum(np.matmul(assignment, seat_affinity.matrix)*np.matmul(guest_affinity.matrix, assignment))

def normalize(x):
    x = x - np.mean(x)
    x = x / np.max(np.abs(x)) * 5
    x = np.exp(x)/np.sum(np.exp(x))
    return x


In [3]:
# visualize
def visualize(solution, seats, guests, seat_coordinates):
    result = np.zeros(tuple(np.max(np.array(list(seat_coordinates.values())), axis=0))).astype(str)
    # result = np.char.replace(result, "0.0", "")
    for g_i in range(len(guests)):
        s_i = solution.genes[g_i] - 1
        seat = list(seats.values())[s_i]
        guest = list(guests.values())[g_i]
        coord = seat_coordinates[seat.name]
        result[coord[0]-1][coord[1]-1] = guest.name
    return result.astype(str)
        

In [4]:
# load guests
df = pd.read_csv('guests.csv', sep=',', header=0)
guest_csv = df.values.astype(str)
guests = {r[0]: Guest(r[0]) for r in guest_csv}
guest_affinity = Affinity.zeros(list(guests.values()))
affinities = [20, 5, 3, 2, 1]
for r in guest_csv:
    affinity_from = guests[r[0]]
    for c in range(1, len(r)):
        if r[c] == 'nan':
            break
        affinity_to = guests[r[c]]
        guest_affinity.set_affinity(affinity_from, affinity_to, affinities[c-1])
        
# load seats
df = pd.read_csv('seats.csv', sep=',', header=0)
seat_csv = df.values.astype(str)
seats = {s: Seat(s) for s in seat_csv.ravel() if s != 'nan'} 
seat_affinity = Affinity.zeros(list(seats.values()))
seat_coordinates = dict()
for i, r in enumerate(seat_csv):
    for j, c in enumerate(r):
        if c == 'nan':
            continue
        seat_coordinates[c] = (i, j)
for s in seats.values():
    coord = seat_coordinates[s.name]
    for s2, coord_2 in seat_coordinates.items():
        s2 = seats[s2]
        if s2 == s:
            continue
        manhattan = np.abs(coord[0] - coord_2[0]) + np.abs(coord[1] - coord_2[1])
        if manhattan <= 4:
            seat_affinity.set_affinity(s, s2, 5-manhattan)


In [19]:
population_size = 2000
generations = 1000
proportion_breed = .8
mutation_prob = .1

all_time_elite = None
all_time_elite_fitness = None
possible_solutions: set()
population = [Chromosome.random(len(seats)) for _ in range(population_size)]
for generation in range(generations):
    population_fitness = [fitness(s, guest_affinity, seat_affinity) for s in population]
    elite_idx = np.argmax(fitness)
    elite = population[elite_idx]
    if all_time_elite is None or population_fitness[elite_idx] > all_time_elite_fitness:
        all_time_elite = elite
        all_time_elite_fitness = population_fitness[elite_idx]
        possible_solutions = set()
    for i in range(len(population)):
        if population_fitness[i] == all_time_elite_fitness:
            possible_solutions.add(population[i])
    print(f"GENERATION: {generation} | Best score: {population_fitness[elite_idx]} | Solutions: {len(possible_solutions)}")
    
    # breed top x%
    breed = np.random.choice(
        population,
        int(len(population) * proportion_breed),
        p=normalize(population_fitness),
        replace=False
    )
    children = list()
    while len(breed) >= 2:
        # choose parents
        parent_1_idx = np.random.randint(0, len(breed))
        parent_1 = breed[parent_1_idx]
        breed = np.delete(breed, parent_1_idx)
        parent_2_idx = np.random.randint(0, len(breed))
        parent_2 = breed[parent_2_idx]
        
        # breed
        breed = np.delete(breed, parent_2_idx)
        mutate = np.random.rand() < mutation_prob
        child1, child2 = parent_1.mate(parent_2, mutate=mutate)
        children.append(child1)
        children.append(child2)
    population = np.append(population, children)
    population_fitness = np.append(population_fitness, [fitness(s, guest_affinity, seat_affinity) for s in children])
        
    # starve off weakest individuals
    population = np.random.choice(population, population_size, p=normalize(population_fitness), replace=False)

print(f"Best all time: {all_time_elite_fitness}")

GENERATION: 0 | Best score: 75.0 | Solutions: 42
GENERATION: 1 | Best score: 80.0 | Solutions: 92
GENERATION: 2 | Best score: 0.0 | Solutions: 146
GENERATION: 3 | Best score: 260.0 | Solutions: 8
GENERATION: 4 | Best score: 80.0 | Solutions: 13
GENERATION: 5 | Best score: 160.0 | Solutions: 22
GENERATION: 6 | Best score: 175.0 | Solutions: 28
GENERATION: 7 | Best score: 160.0 | Solutions: 34
GENERATION: 8 | Best score: 260.0 | Solutions: 47
GENERATION: 9 | Best score: 290.0 | Solutions: 39
GENERATION: 10 | Best score: 240.0 | Solutions: 58


KeyboardInterrupt: 

In [20]:
result = visualize(list(possible_solutions)[0], seats, guests, seat_coordinates)
result = np.char.replace(result, "0.0", "")
np.savetxt("output.csv", result, delimiter=",", fmt="%s")